In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 as cv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,MaxPooling2D,Activation,Flatten,Conv2D,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow import keras
import tensorflow as tf



test_data = pd.read_csv("../input/petfinder-pawpularity-score/test.csv").to_numpy()
train_data = pd.read_csv("../input/petfinder-pawpularity-score/train.csv").to_numpy()

TEST_PATH = '../input/petfinder-pawpularity-score/test'
TRAIN_PATH = '../input/petfinder-pawpularity-score/train'

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D,GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
tabular_train_data_path = '../input/petfinder-pawpularity-score/train.csv'
tabular_test_data_path = '../input/petfinder-pawpularity-score/test.csv'
base_path = '../input/petfinder-pawpularity-score'

In [ ]:
train_df = pd.read_csv(tabular_train_data_path)
train_df.describe()


In [ ]:
train_df.head()

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_df.isna().sum()

In [ ]:
train_df['path'] = base_path + '/train/' + train_df['Id'] + '.jpg'

train_df.head()

In [ ]:
table_input = train_df.iloc[:, 1:-2]
table_input.head(2)

In [ ]:
target = train_df.iloc[:, -2]
target.head(2)

In [ ]:
train_dir = base_path + '/train/'
IMG_SIZE = (224, 224)
train_images = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                   image_size=IMG_SIZE,
                                                                   labels=None,
                                                                   label_mode=None,
                                                                   batch_size=64,
                                                                   shuffle=False)

In [ ]:
AUTO = tf.data.AUTOTUNE

In [ ]:
def preprocess_train(image):
    image = tf.image.random_flip_left_right(image)
    image = image / 255.0
    return image

In [ ]:
base_path = '../input/petfinder-pawpularity-score'

In [ ]:

train_img_data = train_images.map(preprocess_train, num_parallel_calls=AUTO)
train_img_data = train_img_data.prefetch(AUTO)

In [ ]:
image_batch = next(iter(train_img_data))

plt.figure(figsize=(10, 10))
for n in range(25):
    ax = plt.subplot(5, 5, n + 1)
    plt.imshow(image_batch[n])
    plt.axis("off");

In [ ]:
for image in train_img_data.take(1):
    print("Image Size: ", image.shape)
    print("Image max val: ", tf.reduce_max(image))
    print("Image min val: ", tf.reduce_min(image))
    break

In [ ]:


model1 = Sequential([
    
    Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same',input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2,2),strides=2),
    Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same'),
    MaxPool2D(pool_size=(2,2),strides=2),
    Dense(512),
    Flatten(),
    Dense(units=1,activation='softmax'),
])
img_inputs = tf.keras.layers.Input(shape=(224, 224, 3), name='img_input')
model1 = model1(img_inputs)
img_outputs = tf.keras.layers.Dense(512, activation='relu')(model1)
img_model = tf.keras.Model(img_inputs, img_outputs)
img_model.summary()

In [ ]:
tabular_inputs = tf.keras.layers.Input(shape=(12, ), name='tabular_input')
fc1 = tf.keras.layers.Dense(16, activation='relu')(tabular_inputs) 
fc2 = tf.keras.layers.Dense(32, activation='relu')(fc1)
fc3 = tf.keras.layers.Dense(32, activation='relu')(fc2)
tabular_outputs = tf.keras.layers.Dense(512)(fc3)
tabular_model = tf.keras.Model(tabular_inputs, tabular_outputs)
tabular_model.summary()

In [ ]:
concat_models = tf.keras.layers.Concatenate(axis=1)([img_model.output, tabular_model.output])
output = tf.keras.layers.Dense(1, activation='linear')(concat_models)

model = tf.keras.Model(inputs=[img_inputs, tabular_inputs], outputs=output)
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
table_input.shape, target.shape

In [ ]:
table_input_data = tf.data.Dataset.from_tensor_slices(table_input)
table_target_data = tf.data.Dataset.from_tensor_slices(target)

In [ ]:
table_ex_data = table_input_data.batch(64).prefetch(AUTO)

In [ ]:
table_target_ex = table_target_data.batch(64).prefetch(AUTO)

In [ ]:
table_target_ex

In [ ]:
train_input_data = tf.data.Dataset.zip((train_img_data, table_ex_data))
train_data = tf.data.Dataset.zip((train_input_data, table_target_ex))

In [ ]:
train_data

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss = tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')]
)

In [ ]:
history = model.fit(train_data,
                   epochs=10)

In [ ]:
test_df = pd.read_csv(tabular_test_data_path)
test_df.head(10)

In [ ]:
test_dir = base_path + '/test/'

test_images = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                   image_size=IMG_SIZE,
                                                                   labels=None,
                                                                   label_mode=None,
                                                                   batch_size=1,
                                                                   shuffle=False)

In [ ]:
test_input = test_df.iloc[:, 1:]

test_input.head(10)

In [ ]:
test_input = test_input.to_numpy()

In [ ]:
test_input[0]

In [ ]:
test_input_data = tf.data.Dataset.from_tensor_slices(test_input).batch(1).prefetch(AUTO)

test_data = tf.data.Dataset.zip((test_images, test_input_data))

test_data

In [ ]:
img = test_dir + test_df['Id'][0] + '.jpg'

img

In [ ]:
def preprocess_test(image):
    image = tf.io.read_file(image)
    image = tf.io.decode_image(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = image / 255.0
    return image

In [ ]:
image0 = preprocess_test(img)
tf.expand_dims(image0, axis=0).shape

In [ ]:
y_pred = model.predict((tf.expand_dims(image0, axis=0), tf.expand_dims(test_input[0], axis=0)))

In [ ]:
y_pred

In [ ]:
y_pred[0].item()

In [ ]:
# model.save('./pawpularity.mlp')